In [ ]:

%run ../../load_magic/storage.py
%pprint
%matplotlib inline
from itertools import combinations
from math import cos, sin, pi, sqrt, atan
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
%who

In [ ]:

columns_list = ['Red', 'Green', 'Blue']
def colors_dict_to_df(colors_dict):
    rows_list = []
    index_list = []
    for base_name, color_tuple in colors_dict.items():
        row_dict = {}
        index_list.append(base_name)
        for i, color_value in enumerate(columns_list):
            row_dict[color_value] = color_tuple[i]
        rows_list.append(row_dict)
    df = pd.DataFrame(rows_list, columns=columns_list, index=index_list)
    
    return df

In [ ]:

def show_3d_plot(three_d_df, z_column='Red', x_column='Green', y_column='Blue'):
    fig = plt.figure(figsize=(18, 8))
    ax = fig.add_subplot(111, projection='3d', autoscale_on=True)
    xlabel_text = ax.set_xlabel(x_column)
    ylabel_text = ax.set_ylabel(y_column)
    zlabel_text = ax.set_zlabel(z_column)
    columns_list = [x_column, y_column, z_column]
    df = three_d_df[columns_list].dropna(axis='index', how='any')
    pca_ndarray = df.values
    path_collection = ax.scatter(pca_ndarray[:, 0], pca_ndarray[:, 1],
                                 pca_ndarray[:, 2], alpha=0.75, c=df.index)
    title_text = 'Scatterplot of the {}, {}, and {} Data'
    text_obj = ax.set_title(title_text.format(x_column, y_column, z_column))

In [ ]:

def distance_from_white(old_tuple):
    green_diff = 1.0 - old_tuple[0]
    blue_diff = 1.0 - old_tuple[1]
    red_diff = 1.0 - old_tuple[2]
    
    return sqrt(green_diff**2 + blue_diff**2 + red_diff**2)

In [ ]:

def distance_from_black(old_tuple):
    
    return sqrt(old_tuple[0]**2 + old_tuple[1]**2 + old_tuple[2]**2)

In [ ]:

def distance_from_red(old_tuple):
    red_diff = 1.0 - old_tuple[2]
    
    return sqrt(old_tuple[0]**2 + old_tuple[1]**2 + red_diff**2)

In [ ]:

def distance_from_green(old_tuple):
    green_diff = 1.0 - old_tuple[0]
    
    return sqrt(green_diff**2 + old_tuple[1]**2 + old_tuple[2]**2)

In [ ]:

def distance_from_blue(old_tuple):
    blue_diff = 1.0 - old_tuple[1]
    
    return sqrt(old_tuple[0]**2 + blue_diff**2 + old_tuple[2]**2)

In [ ]:

def distance_from_magenta(old_tuple):
    blue_diff = 1.0 - old_tuple[1]
    red_diff = 1.0 - old_tuple[2]
    
    return sqrt(old_tuple[0]**2 + blue_diff**2 + red_diff**2)

In [ ]:

def distance_from_yellow(old_tuple):
    green_diff = 1.0 - old_tuple[0]
    red_diff = 1.0 - old_tuple[2]
    
    return sqrt(green_diff**2 + old_tuple[1]**2 + red_diff**2)

In [ ]:

def distance_from_cyan(old_tuple):
    green_diff = 1.0 - old_tuple[0]
    blue_diff = 1.0 - old_tuple[1]
    
    return sqrt(green_diff**2 + blue_diff**2 + old_tuple[2]**2)

In [ ]:

def get_distance_df(colors_df):
    rows_list = []
    columns_list = ['distance_from_white', 'distance_from_black', 'distance_from_red', 'distance_from_green', 'distance_from_blue',
                    'distance_from_magenta', 'distance_from_yellow', 'distance_from_cyan']
    index_list = []
    for row_index, row_series in colors_df.iterrows():
        green_value = row_series['Green']
        blue_value = row_series['Blue']
        red_value = row_series['Red']
        row_tuple = (green_value, blue_value, red_value)
        row_dict = {}
        row_dict['distance_from_white'] = distance_from_white(row_tuple)
        row_dict['distance_from_black'] = distance_from_black(row_tuple)
        row_dict['distance_from_red'] = distance_from_red(row_tuple)
        row_dict['distance_from_green'] = distance_from_green(row_tuple)
        row_dict['distance_from_blue'] = distance_from_blue(row_tuple)
        row_dict['distance_from_magenta'] = distance_from_magenta(row_tuple)
        row_dict['distance_from_yellow'] = distance_from_yellow(row_tuple)
        row_dict['distance_from_cyan'] = distance_from_cyan(row_tuple)
        rows_list.append(row_dict)
        index_list.append(row_index)
    distance_df = pd.DataFrame(rows_list, columns=columns_list, index=index_list)
    
    return distance_df

In [ ]:

full_corner_list = ['white', 'black', 'red', 'green', 'blue', 'magenta', 'yellow', 'cyan']
def get_face_set_list(combinations_list):
    combs_obj = combinations(combinations_list, 3)
    face_set_list = []
    for color_tuple in combs_obj:
        face_set_list.append(set(color_tuple))
    
    return face_set_list

In [ ]:

def plot_colortable(colors_dict, title, sort_colors=True, emptycols=0):
    if len(colors_dict):

        cell_width = 212
        cell_height = 22
        swatch_width = 48
        margin = 12
        topmargin = 40

        # Sort colors_dict by hue, saturation, value and name.
        if sort_colors is True:
            by_hsv = sorted((tuple(mcolors.rgb_to_hsv(mcolors.to_rgb(color))),
                             name)
                            for name, color in colors_dict.items())
            names = [name for hsv, name in by_hsv]
        else:
            names = list(colors_dict)

        n = len(names)
        ncols = 4 - emptycols
        nrows = n // ncols + int(n % ncols > 0)

        width = cell_width * 4 + 2 * margin
        height = cell_height * nrows + margin + topmargin
        dpi = 72

        fig, ax = plt.subplots(figsize=(width / dpi, height / dpi), dpi=dpi)
        fig.subplots_adjust(margin/width, margin/height,
                            (width-margin)/width, (height-topmargin)/height)
        ax.set_xlim(0, cell_width * 4)
        ax.set_ylim(cell_height * (nrows-0.5), -cell_height/2.)
        ax.yaxis.set_visible(False)
        ax.xaxis.set_visible(False)
        ax.set_axis_off()
        ax.set_title(title, fontsize=24, loc="left", pad=10)

        for i, name in enumerate(names):
            row = i % nrows
            col = i // nrows
            y = row * cell_height

            swatch_start_x = cell_width * col
            swatch_end_x = cell_width * col + swatch_width
            text_pos_x = cell_width * col + swatch_width + 7

            ax.text(text_pos_x, y, name, fontsize=14,
                    horizontalalignment='left',
                    verticalalignment='center')

            ax.hlines(y, swatch_start_x, swatch_end_x,
                      color=colors_dict[name], linewidth=18)

In [ ]:

# Hue, Saturation, Value
def get_hsv_dict(colors_dict):
    
    return {name: tuple(mcolors.rgb_to_hsv(mcolors.to_rgb(color))) for name, color in colors_dict.items()}

In [ ]:

def show_face_proximity(distance_df):
    for row_index, row_series in distance_df.iterrows():
        tuple_list = sorted(row_series.to_dict().items(), key=lambda x: x[1])
        if tuple_list[0][1] == 0.0:
            print('{} is in the {} corner'.format(row_index, tuple_list[0][0].split('_')[2]))
        else:
            corners_list = tuple_list[:3]
            face_set = set([corners_list[0][0].split('_')[2], corners_list[1][0].split('_')[2], corners_list[2][0].split('_')[2]])
            if face_set in kryg_face_set_list:
                print('{} is nearest the black-red-yellow-green face'.format(row_index))
            elif face_set in krmb_face_set_list:
                print('{} is nearest the black-red-magenta-blue face'.format(row_index))
            elif face_set in kbcg_face_set_list:
                print('{} is nearest the black-blue-cyan-green face'.format(row_index))
            elif face_set in wcgy_face_set_list:
                print('{} is nearest the white-cyan-green-yellow face'.format(row_index))
            elif face_set in wcbm_face_set_list:
                print('{} is nearest the white-cyan-blue-magenta face'.format(row_index))
            elif face_set in wyrm_face_set_list:
                print('{} is nearest the white-yellow-red-magenta face'.format(row_index))
            else:
                print('{} is nearest the {} face'.format(row_index, '-'.join(list(face_set))))

In [ ]:

def get_face_dictionary(distance_df):
    face_dictionary = {}
    for row_index, row_series in distance_df.iterrows():
        tuple_list = sorted(row_series.to_dict().items(), key=lambda x: x[1])
        if tuple_list[0][1] == 0.0:
            face_dictionary[row_index] = tuple_list[0][0].split('_')[2]
        else:
            corners_list = tuple_list[:3]
            face_set = set([corners_list[0][0].split('_')[2], corners_list[1][0].split('_')[2], corners_list[2][0].split('_')[2]])
            if face_set in kryg_face_set_list:
                face_dictionary[row_index] = 'black-red-yellow-green'
            elif face_set in krmb_face_set_list:
                face_dictionary[row_index] = 'black-red-magenta-blue'
            elif face_set in kbcg_face_set_list:
                face_dictionary[row_index] = 'black-blue-cyan-green'
            elif face_set in wcgy_face_set_list:
                face_dictionary[row_index] = 'white-cyan-green-yellow'
            elif face_set in wcbm_face_set_list:
                face_dictionary[row_index] = 'white-cyan-blue-magenta'
            elif face_set in wyrm_face_set_list:
                face_dictionary[row_index] = 'white-yellow-red-magenta'
            else:
                face_dictionary[row_index] = '-'.join(list(face_set))
    
    return face_dictionary

In [ ]:

def distance_from_kryg_face(old_tuple):
    green_diff = 0.5 - old_tuple[0]
    red_diff = 0.5 - old_tuple[2]
    
    return sqrt(green_diff**2 + old_tuple[1]**2 + red_diff**2)

In [ ]:

def distance_from_krmb_face(old_tuple):
    blue_diff = 0.5 - old_tuple[1]
    red_diff = 0.5 - old_tuple[2]
    
    return sqrt(old_tuple[0]**2 + blue_diff**2 + red_diff**2)

In [ ]:

def distance_from_kbcg_face(old_tuple):
    green_diff = 0.5 - old_tuple[0]
    blue_diff = 0.5 - old_tuple[1]
    
    return sqrt(green_diff**2 + blue_diff**2 + old_tuple[2]**2)

In [ ]:

def distance_from_wcgy_face(old_tuple):
    green_diff = 1.0 - old_tuple[0]
    blue_diff = 0.5 - old_tuple[1]
    red_diff = 0.5 - old_tuple[2]
    
    return sqrt(green_diff**2 + blue_diff**2 + red_diff**2)

In [ ]:

def distance_from_wcbm_face(old_tuple):
    green_diff = 0.5 - old_tuple[0]
    blue_diff = 1.0 - old_tuple[1]
    red_diff = 0.5 - old_tuple[2]
    
    return sqrt(green_diff**2 + blue_diff**2 + red_diff**2)

In [ ]:

def distance_from_wyrm_face(old_tuple):
    green_diff = 0.5 - old_tuple[0]
    blue_diff = 0.5 - old_tuple[1]
    red_diff = 1.0 - old_tuple[2]
    
    return sqrt(green_diff**2 + blue_diff**2 + red_diff**2)

In [ ]:

def display_test_colors(test_list, saeculum_title, face_title, nearness_str='far from', color_dict=mcolors.XKCD_COLORS, color_title='XKCD'):
    print(test_list)
    plot_colortable(colors_dict={name: color for name, color in color_dict.items() if name in [name for distance, name in test_list]},
                    title='{} {} Colors, {} the {} Face'.format(color_title, saeculum_title, nearness_str, face_title),
                    sort_colors=True, emptycols=0)

In [ ]:

kryg_face_set_list = get_face_set_list(['black', 'red', 'yellow', 'green'])
krmb_face_set_list = get_face_set_list(['black', 'red', 'magenta', 'blue'])
kbcg_face_set_list = get_face_set_list(['black', 'blue', 'cyan', 'green'])
wcgy_face_set_list = get_face_set_list(['white', 'cyan', 'green', 'yellow'])
wcbm_face_set_list = get_face_set_list(['white', 'cyan', 'blue', 'magenta'])
wyrm_face_set_list = get_face_set_list(['white', 'yellow', 'red', 'magenta'])


# XKCD

In [ ]:

colors_dict = {name: mcolors.to_rgb(color) for name, color in mcolors.XKCD_COLORS.items()}
xkcd_colors_df = colors_dict_to_df(colors_dict)
show_3d_plot(xkcd_colors_df)

In [ ]:

xkcd_distance_df = get_distance_df(xkcd_colors_df)
xkcd_face_dictionary = get_face_dictionary(xkcd_distance_df)
#show_face_proximity(xkcd_distance_df)

In [ ]:

np.unique(list(xkcd_face_dictionary.values()))

In [ ]:

patriline_df = load_object('patriline_df')
min_year = patriline_df['Year of Birth'].min()
generations_df = load_object('generations_df')
match_series = (generations_df['birth_year_begin'] >= min_year)
generations_df[match_series]['saeculum_name'].unique()

In [ ]:

for saeculum_name in generations_df[match_series]['saeculum_name'].unique():
    saeculum_name = '_'.join(saeculum_name.lower().split(' '))
    print('xkcd_{}_list = []'.format(saeculum_name))

In [ ]:

xkcd_reformation_list = []
xkcd_new_world_list = []
xkcd_revolutionary_list = []
xkcd_civil_war_list = []
xkcd_great_power_list = []
xkcd_millennial_list = []
for key, value in xkcd_face_dictionary.items():
    if value == 'black-blue-cyan-green':
        xkcd_reformation_list.append(key)
    elif value == 'black-red-magenta-blue':
        xkcd_new_world_list.append(key)
    elif value == 'black-red-yellow-green':
        xkcd_revolutionary_list.append(key)
    elif value == 'white-cyan-blue-magenta':
        xkcd_civil_war_list.append(key)
    elif value == 'white-cyan-green-yellow':
        xkcd_great_power_list.append(key)
    elif value == 'white-yellow-red-magenta':
        xkcd_millennial_list.append(key)

In [ ]:

xkcd_kbcg_reformation_dict = {name: distance_from_kbcg_face(mcolors.to_rgb(color)) for name,
                              color in mcolors.XKCD_COLORS.items() if name in xkcd_reformation_list}
color_tuple_list = sorted((distance, name) for name, distance in xkcd_kbcg_reformation_dict.items())

In [ ]:

test_list = color_tuple_list[:4]
saeculum_title = 'Reformation'
face_title = 'KBCG'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

test_list = color_tuple_list[-4:]
saeculum_title = 'Reformation'
face_title = 'KBCG'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

xkcd_krmb_new_world_dict = {name: distance_from_krmb_face(mcolors.to_rgb(color)) for name,
                            color in mcolors.XKCD_COLORS.items() if name in xkcd_new_world_list}
color_tuple_list = sorted((distance, name) for name, distance in xkcd_krmb_new_world_dict.items())

In [ ]:

test_list = color_tuple_list[:4]
saeculum_title = 'New World'
face_title = 'KRMB'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

test_list = color_tuple_list[-4:]
saeculum_title = 'New World'
face_title = 'KRMB'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

xkcd_kryg_revolutionary_dict = {name: distance_from_kryg_face(mcolors.to_rgb(color)) for name,
                                color in mcolors.XKCD_COLORS.items() if name in xkcd_revolutionary_list}
color_tuple_list = sorted((distance, name) for name, distance in xkcd_kryg_revolutionary_dict.items())

In [ ]:

test_list = color_tuple_list[:4]
saeculum_title = 'Revolutionary'
face_title = 'KRYG'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

test_list = color_tuple_list[-4:]
saeculum_title = 'Revolutionary'
face_title = 'KRYG'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

xkcd_wcbm_civil_war_dict = {name: distance_from_wcbm_face(mcolors.to_rgb(color)) for name,
                            color in mcolors.XKCD_COLORS.items() if name in xkcd_civil_war_list}
color_tuple_list = sorted((distance, name) for name, distance in xkcd_wcbm_civil_war_dict.items())

In [ ]:

test_list = color_tuple_list[:4]
saeculum_title = 'Civil War'
face_title = 'WCBM'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

test_list = color_tuple_list[-4:]
saeculum_title = 'Civil War'
face_title = 'WCBM'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

xkcd_wcgy_great_power_dict = {name: distance_from_wcgy_face(mcolors.to_rgb(color)) for name,
                              color in mcolors.XKCD_COLORS.items() if name in xkcd_great_power_list}
color_tuple_list = sorted((distance, name) for name, distance in xkcd_wcgy_great_power_dict.items())

In [ ]:

test_list = color_tuple_list[:4]
saeculum_title = 'Great Power'
face_title = 'WCGY'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

test_list = color_tuple_list[-4:]
saeculum_title = 'Great Power'
face_title = 'WCGY'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

xkcd_wyrm_millennial_dict = {name: distance_from_wyrm_face(mcolors.to_rgb(color)) for name,
                             color in mcolors.XKCD_COLORS.items() if name in xkcd_millennial_list}
color_tuple_list = sorted((distance, name) for name, distance in xkcd_wyrm_millennial_dict.items())

In [ ]:

test_list = color_tuple_list[:4]
saeculum_title = 'Millenial'
face_title = 'WYRM'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

test_list = color_tuple_list[-4:]
saeculum_title = 'Millenial'
face_title = 'WYRM'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

test_list = color_tuple_list[:20:5]
saeculum_title = 'Millenial'
face_title = 'WYRM'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.XKCD_COLORS.items() if name in xkcd_reformation_list},
                title='XKCD Reformation Colors', sort_colors=True, emptycols=0)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.XKCD_COLORS.items() if name in xkcd_new_world_list},
                title='XKCD New World Colors', sort_colors=True, emptycols=0)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.XKCD_COLORS.items() if name in xkcd_revolutionary_list},
                title='XKCD Revolutionary Colors', sort_colors=True, emptycols=0)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.XKCD_COLORS.items() if name in xkcd_civil_war_list},
                title='XKCD Civil War Colors', sort_colors=True, emptycols=0)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.XKCD_COLORS.items() if name in xkcd_great_power_list},
                title='XKCD Great Power Colors', sort_colors=True, emptycols=0)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.XKCD_COLORS.items() if name in xkcd_millennial_list},
                title='XKCD Millennial Colors', sort_colors=True, emptycols=0)


# CSS4

In [ ]:

colors_dict = {name: mcolors.to_rgb(color) for name, color in mcolors.CSS4_COLORS.items()}
css4_colors_df = colors_dict_to_df(colors_dict)
show_3d_plot(css4_colors_df)

In [ ]:

css4_reformation_list = []
css4_new_world_list = []
css4_revolutionary_list = []
css4_civil_war_list = []
css4_great_power_list = []
css4_millennial_list = []
css4_distance_df = get_distance_df(css4_colors_df)
css4_face_dictionary = get_face_dictionary(css4_distance_df)
for key, value in css4_face_dictionary.items():
    if value == 'black-blue-cyan-green':
        css4_reformation_list .append(key)
    elif value == 'black-red-magenta-blue':
        css4_new_world_list .append(key)
    elif value == 'black-red-yellow-green':
        css4_revolutionary_list .append(key)
    elif value == 'white-cyan-blue-magenta':
        css4_civil_war_list .append(key)
    elif value == 'white-cyan-green-yellow':
        css4_great_power_list .append(key)
    elif value == 'white-yellow-red-magenta':
        css4_millennial_list .append(key)

In [ ]:

css4_kbcg_reformation_dict = {name: distance_from_kbcg_face(mcolors.to_rgb(color)) for name,
                              color in mcolors.CSS4_COLORS.items() if name in css4_reformation_list}

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.CSS4_COLORS.items() if name in css4_reformation_list},
                title='CSS4 Reformation Colors', sort_colors=True, emptycols=1)

In [ ]:

color_tuple_list = sorted((distance, name) for name, distance in css4_kbcg_reformation_dict.items())
test_list = color_tuple_list[:8]
saeculum_title = 'Reformation'
face_title = 'KBCG'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

color_tuple_list = sorted((distance, name) for name, distance in css4_kbcg_reformation_dict.items())
test_list = color_tuple_list[-4:]
saeculum_title = 'Reformation'
face_title = 'KBCG'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

css4_krmb_new_world_dict = {name: distance_from_krmb_face(mcolors.to_rgb(color)) for name,
                            color in mcolors.CSS4_COLORS.items() if name in css4_new_world_list}

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.CSS4_COLORS.items() if name in css4_new_world_list},
                title='CSS4 New World Colors', sort_colors=True, emptycols=1)

In [ ]:

color_tuple_list = sorted((distance, name) for name, distance in css4_krmb_new_world_dict.items())
test_list = color_tuple_list[:12]
saeculum_title = 'New World'
face_title = 'KRMB'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

color_tuple_list = sorted((distance, name) for name, distance in css4_krmb_new_world_dict.items())
test_list = color_tuple_list[-4:]
saeculum_title = 'New World'
face_title = 'KRMB'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

css4_kryg_revolutionary_dict = {name: distance_from_kryg_face(mcolors.to_rgb(color)) for name,
                                color in mcolors.CSS4_COLORS.items() if name in css4_revolutionary_list}

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.CSS4_COLORS.items() if name in css4_revolutionary_list},
                title='CSS4 Revolutionary Colors', sort_colors=True, emptycols=1)

In [ ]:

color_tuple_list = sorted((distance, name) for name, distance in css4_kryg_revolutionary_dict.items())
test_list = color_tuple_list[:12]
saeculum_title = 'Revolutionary'
face_title = 'KRYG'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

color_tuple_list = sorted((distance, name) for name, distance in css4_kryg_revolutionary_dict.items())
test_list = color_tuple_list[-4:]
saeculum_title = 'Revolutionary'
face_title = 'KRYG'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

css4_wcbm_civil_war_dict = {name: distance_from_wcbm_face(mcolors.to_rgb(color)) for name,
                            color in mcolors.CSS4_COLORS.items() if name in css4_civil_war_list}

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.CSS4_COLORS.items() if name in css4_civil_war_list},
                title='CSS4 Civil War Colors', sort_colors=True, emptycols=1)

In [ ]:

color_tuple_list = sorted((distance, name) for name, distance in css4_wcbm_civil_war_dict.items())
test_list = color_tuple_list[:4]
saeculum_title = 'Civil War'
face_title = 'WCBM'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

color_tuple_list = sorted((distance, name) for name, distance in css4_wcbm_civil_war_dict.items())
test_list = color_tuple_list[:12]
saeculum_title = 'Civil War'
face_title = 'WCBM'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

color_tuple_list = sorted((distance, name) for name, distance in css4_wcbm_civil_war_dict.items())
test_list = color_tuple_list[-12:]
saeculum_title = 'Civil War'
face_title = 'WCBM'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

css4_wcgy_great_power_dict = {name: distance_from_wcgy_face(mcolors.to_rgb(color)) for name,
                              color in mcolors.CSS4_COLORS.items() if name in css4_great_power_list}
color_tuple_list = sorted((distance, name) for name, distance in css4_wcgy_great_power_dict.items())

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.CSS4_COLORS.items() if name in css4_great_power_list},
                title='CSS4 Great Power Colors', sort_colors=True, emptycols=1)

In [ ]:

test_list = color_tuple_list[:4]
saeculum_title = 'Great Power'
face_title = 'WCGY'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

test_list = color_tuple_list[-4:]
saeculum_title = 'Great Power'
face_title = 'WCGY'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

css4_wyrm_millennial_dict = {name: distance_from_wyrm_face(mcolors.to_rgb(color)) for name,
                             color in mcolors.CSS4_COLORS.items() if name in css4_millennial_list}
color_tuple_list = sorted((distance, name) for name, distance in css4_wyrm_millennial_dict.items())

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.CSS4_COLORS.items() if name in css4_millennial_list},
                title='CSS4 Millennial Colors', sort_colors=True, emptycols=1)

In [ ]:

test_list = color_tuple_list[:4]
saeculum_title = 'Millenial'
face_title = 'WYRM'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

test_list = color_tuple_list[-4:]
saeculum_title = 'Millenial'
face_title = 'WYRM'
nearness_str = 'far from'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

test_list = color_tuple_list[:20:5]
saeculum_title = 'Millenial'
face_title = 'WYRM'
nearness_str = 'close to'
display_test_colors(test_list, saeculum_title, face_title, nearness_str, color_dict=mcolors.CSS4_COLORS, color_title='CSS4')

In [ ]:

colors_dict = {name: mcolors.to_rgb(color) for name, color in mcolors.BASE_COLORS.items()}
base_colors_df = colors_dict_to_df(colors_dict)
show_3d_plot(base_colors_df)

In [ ]:

base_reformation_list = []
base_new_world_list = []
base_revolutionary_list = []
base_civil_war_list = []
base_great_power_list = []
base_millennial_list = []
base_distance_df = get_distance_df(base_colors_df)
base_face_dictionary = get_face_dictionary(base_distance_df)
for key, value in base_face_dictionary.items():
    if value == 'black-blue-cyan-green':
        base_reformation_list .append(key)
    elif value == 'black-red-magenta-blue':
        base_new_world_list .append(key)
    elif value == 'black-red-yellow-green':
        base_revolutionary_list .append(key)
    elif value == 'white-cyan-blue-magenta':
        base_civil_war_list .append(key)
    elif value == 'white-cyan-green-yellow':
        base_great_power_list .append(key)
    elif value == 'white-yellow-red-magenta':
        base_millennial_list .append(key)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.BASE_COLORS.items() if name in base_reformation_list},
                title='Base Reformation Colors', sort_colors=True, emptycols=1)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.BASE_COLORS.items() if name in base_new_world_list},
                title='Base New_world Colors', sort_colors=True, emptycols=1)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.BASE_COLORS.items() if name in base_revolutionary_list},
                title='Base Revolutionary Colors', sort_colors=True, emptycols=1)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.BASE_COLORS.items() if name in base_civil_war_list},
                title='Base Civil_war Colors', sort_colors=True, emptycols=1)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.BASE_COLORS.items() if name in base_great_power_list},
                title='Base Great_power Colors', sort_colors=True, emptycols=1)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.BASE_COLORS.items() if name in base_millennial_list},
                title='Base Millennial Colors', sort_colors=True, emptycols=1)

In [ ]:

show_face_proximity(base_distance_df)

In [ ]:

colors_dict = {name: mcolors.to_rgb(color) for name, color in mcolors.TABLEAU_COLORS.items()}
tab_colors_df = colors_dict_to_df(colors_dict)
show_3d_plot(tab_colors_df)

In [ ]:

tab_reformation_list = []
tab_new_world_list = []
tab_revolutionary_list = []
tab_civil_war_list = []
tab_great_power_list = []
tab_millennial_list = []
tab_distance_df = get_distance_df(tab_colors_df)
tab_face_dictionary = get_face_dictionary(tab_distance_df)
for key, value in tab_face_dictionary.items():
    if value == 'black-blue-cyan-green':
        tab_reformation_list .append(key)
    elif value == 'black-red-magenta-blue':
        tab_new_world_list .append(key)
    elif value == 'black-red-yellow-green':
        tab_revolutionary_list .append(key)
    elif value == 'white-cyan-blue-magenta':
        tab_civil_war_list .append(key)
    elif value == 'white-cyan-green-yellow':
        tab_great_power_list .append(key)
    elif value == 'white-yellow-red-magenta':
        tab_millennial_list .append(key)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.TABLEAU_COLORS.items() if name in tab_reformation_list},
                title='Tableau Reformation Colors', sort_colors=True, emptycols=1)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.TABLEAU_COLORS.items() if name in tab_new_world_list},
                title='Tableau New World Colors', sort_colors=True, emptycols=1)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.TABLEAU_COLORS.items() if name in tab_revolutionary_list},
                title='Tableau Revolutionary Colors', sort_colors=True, emptycols=1)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.TABLEAU_COLORS.items() if name in tab_civil_war_list},
                title='Tableau Civil War Colors', sort_colors=True, emptycols=1)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.TABLEAU_COLORS.items() if name in tab_great_power_list},
                title='Tableau Great Power Colors', sort_colors=True, emptycols=1)

In [ ]:

plot_colortable(colors_dict={name: color for name, color in mcolors.TABLEAU_COLORS.items() if name in tab_millennial_list},
                title='Tableau Millennial Colors', sort_colors=True, emptycols=1)

In [ ]:

X = xkcd_colors_df.values
X2D = PCA(n_components=2).fit_transform(X)
fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal')

ax.plot(X2D[:, 0], X2D[:, 1], 'k+')
ax.plot(X2D[:, 0], X2D[:, 1], 'k.')
ax.plot([0], [0], 'ko')
ax.arrow(0, 0, 0, 1, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
ax.arrow(0, 0, 1, 0, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
ax.set_xlabel('$z_1$', fontsize=18)
ax.set_ylabel('$z_2$', fontsize=18, rotation=0)
ax.axis([-1.5, 1.3, -1.2, 1.2])
ax.grid(True)

In [ ]:

eps = 0.09
min_samples = 1
db = DBSCAN(eps=eps, min_samples=min_samples).fit(X2D)
labels_ndarray = db.labels_
unique_labels = np.unique(labels_ndarray)
unique_labels_count = len(unique_labels)
if (unique_labels_count > 4) and (unique_labels_count < 8):
    print('np.unique(DBSCAN(eps={}, min_samples={}).fit(pca_ndarray).labels_) = {}'.format(eps, min_samples, unique_labels))

In [ ]:

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, autoscale_on=True)
cmap = plt.get_cmap('viridis_r')
path_collection = ax.scatter(X2D[:, 0], X2D[:, 1],
                             c=labels_ndarray, edgecolors=(0, 0, 0), cmap=cmap)
title_text = 'Dimension Reduced Scatterplot of the XKCD Colors'
text = plt.title(title_text)
xticklabel_obj = plt.xticks([])
yticklabel_obj = plt.yticks([])

In [ ]:

# Optionally plot the XKCD colors (Caution: will produce large figure)
file_name = 'XKCD_Colors.png'
png_folder = os.path.join(SAVES_FOLDER, 'png')
file_path = os.path.join(png_folder, file_name)
xkcd_fig = plot_colortable(mcolors.XKCD_COLORS, "XKCD Colors")
xkcd_fig.savefig(file_path)